<a href="https://colab.research.google.com/github/marsggbo/AutoMLDemos/blob/master/ch3/EA-NAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 搜索空间

In [1]:
import random
import torch
from torch import nn

# 搜索空间总共包含3**6=729个候选模型
spaces = {
    'channels_1': [16, 32, 64],
    'channels_2': [16, 32, 64],
    'channels_3': [16, 32, 64],
    'kernel_size_1': [3, 5, 7],
    'kernel_size_2': [3, 5, 7],
    'kernel_size_3': [3, 5, 7],
} 

# 模型构造函数
def build_model(c1, c2, c3, ks1, ks2, ks3):
    return nn.Sequential(
        nn.Conv2d(3, c1, kernel_size=ks1, stride=1, padding=ks1//2),
        nn.BatchNorm2d(c1), nn.ReLU(),
        nn.Conv2d(c1, c2, kernel_size=ks2, stride=2, padding=ks2//2),
        nn.BatchNorm2d(c2), nn.ReLU(),
        nn.Conv2d(c2, c3, kernel_size=ks3, stride=2, padding=ks3//2),
        nn.BatchNorm2d(c3), nn.ReLU(),
        nn.AdaptiveAvgPool2d(1), nn.Flatten(1),
        nn.Linear(c3, 10),  # 假设最后一层是10类分类任务
    )

def evaluate_model(model):
    accuracy = torch.rand(1).item()
    # 省略模型评估过程，根据具体任务进行模型评估，并返回性能指标
    return accuracy

d:\miniconda3\envs\hyperbox\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. 进化操作
- 随机采样
- 选择
- 交叉
- 变异
- 更新

In [22]:
# 随机采样模型编码
def sample_encoding():
    encoding = [random.choice(space) for space in spaces.values()]
    return encoding

# 交叉操作
def crossover(parent1, parent2):
    crossover_point = len(parent1) // 2
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# 变异操作
def mutation(individual, mutation_rate):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            max_value = len(list(spaces.values())[i])
            individual[i] = random.randint(0, max_value)
    return individual

# 适应度函数（用于选择操作）
def fitness_function(individual):
    return evaluate_model(individual)


# 3. 进化算法主函数

In [23]:

# 遗传算法
def genetic_algorithm(population, num_generations, exist_ratio, mutation_ratio, crossover_ratio, mutation_rate):
    assert exist_ratio + mutation_ratio + crossover_ratio < 1, "The sum of ratios must be equal to 1."

    pop_size = len(population)
    num_existing = int(exist_ratio * pop_size)
    num_crossover = int(crossover_ratio * pop_size)
    num_mutated = int(mutation_ratio * pop_size)
    num_random = pop_size - num_existing - num_crossover - num_mutated

    for gen in range(num_generations):
        # 计算适应度并对种群进行排序
        fitness_scores = [(individual, fitness_function(individual)) for individual in population]
        fitness_scores.sort(key=lambda x: x[1], reverse=True)
        print(f"Evolution generation {gen + 1}: best accuracy {fitness_scores[0][1]}")

        # 保留现有的模型
        new_population = [ind for ind, _ in fitness_scores[:num_existing]]

        # 生成经过变异的模型
        for _ in range(num_mutated):
            parent = random.choice(population)
            new_population.append(mutation(parent, mutation_rate))
            
        # 通过交叉和变异生成新模型
        for _ in range(num_crossover // 2):
            parent1 = random.choice(population)
            parent2 = random.choice(population)
            child1, child2 = crossover(parent1, parent2)
            new_population += [mutation(child1, mutation_rate), mutation(child2, mutation_rate)]

        # 随机生成新模型
        for _ in range(num_random):
            new_population.append(sample_encoding())

        population = new_population

    fitness_scores = [(individual, fitness_function(individual)) for individual in population]
    fitness_scores.sort(key=lambda x: x[1], reverse=True)
    return fitness_scores[0]

# 运行遗传算法
population_size = 100
individual_length = 10
min_value = 0
max_value = 100
num_generations = 100
exist_ratio = 0.4
mutation_ratio = 0.2
crossover_ratio = 0.2
mutation_rate = 0.1

initial_population = [sample_encoding() for _ in range(population_size)]
best_individual, best_acc = genetic_algorithm(initial_population, num_generations, exist_ratio, mutation_ratio, crossover_ratio, mutation_rate)
print(f"Best individual {best_individual} with acc {best_acc:.4f}")

Evolution generation 1: best accuracy 0.9932825565338135
Evolution generation 2: best accuracy 0.9928039312362671
Evolution generation 3: best accuracy 0.9963374137878418
Evolution generation 4: best accuracy 0.9874873757362366
Evolution generation 5: best accuracy 0.9934592247009277
Evolution generation 6: best accuracy 0.994558572769165
Evolution generation 7: best accuracy 0.9971825480461121
Evolution generation 8: best accuracy 0.9852906465530396
Evolution generation 9: best accuracy 0.9830977916717529
Evolution generation 10: best accuracy 0.9864993691444397
Evolution generation 11: best accuracy 0.9937055706977844
Evolution generation 12: best accuracy 0.9762774109840393
Evolution generation 13: best accuracy 0.9997625350952148
Evolution generation 14: best accuracy 0.9951463937759399
Evolution generation 15: best accuracy 0.9953106641769409
Evolution generation 16: best accuracy 0.9857903718948364
Evolution generation 17: best accuracy 0.9982297420501709
Evolution generation 18: